In [1]:
# -*- coding: utf-8 -*-
# 参考代码：https://github.com/Winteee/KerasTest/blob/master/KerasLSTM.py
# 这是代码的文字解释：http://spaces.ac.cn/index.php/archives/3414/comment-page-1

import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词
import jieba.analyse
 
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU

Using TensorFlow backend.


In [2]:
print("————————语料预处理：数据导入————————")
reviewfile=pd.read_csv('review30000.csv') #读取训练语料完毕

reviewdata = pd.DataFrame()
reviewdata["reviewbody"]= reviewfile["b.reviewbody"]
reviewdata["score1"]= reviewfile["口味评分"]
reviewdata["score2"]= reviewfile["环境评分"]
reviewdata["score3"]= reviewfile["服务评分"]
reviewdata = reviewdata.dropna(axis=0,how='any') #去掉所有有缺失值的行
print("reviewdata的数据格式为：",reviewdata.columns,reviewdata.index)
print("——————————over——————————")


————————语料预处理：数据导入————————


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


reviewdata的数据格式为： Index(['reviewbody', 'score1', 'score2', 'score3'], dtype='object') Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            29990, 29991, 29992, 29993, 29994, 29995, 29996, 29997, 29998,
            29999],
           dtype='int64', length=30000)
——————————over——————————


In [3]:
def cutwords(review_data):
    cw = lambda x: list(jieba.cut(x)) #定义分词函数
    review_data['review_cut_words'] = review_data["reviewbody"].apply(cw)
    print("——————————分词结束——————————")
    return review_data

def cleanwords(review_data):
    delwordslist = "，/,/。/的/！/～/、/（/）/ /./。/吧".split("/")
    if "review_cut_words" in review_data.columns:
        for i in review_data.index:
            one = review_data.review_cut_words[i]
            for k in one:
                if k in delwordslist:
                    one.remove(k)
            i +=1
        return review_data
    
    elif "review_cut_words" not in review_data.columns:
        print("cleanwords运行出错，请先运行cutwords函数以获取review_cut_words")

def countwords(review_data):
    w = []
    for i in review_data["review_cut_words"]:
        w.extend(i)
    reviewdict = pd.DataFrame(pd.Series(w).value_counts(),columns = ["counts"])
    print("——————————词频统计结束——————————")
    return reviewdict

def arraywords(review_data,reviewdict):
    reviewdict['id']=list(range(1,len(reviewdict)+1))
    get_sent = lambda x: list(reviewdict["counts"][x])
    maxlen = 50
    review_data['sent'] = review_data['review_cut_words'].apply(get_sent) 
    review_data['sent'] = list(sequence.pad_sequences(review_data['sent'], maxlen=maxlen))
    print("——————————词向量化结束——————————")
    return review_data

#def markwords(review_data):
 #   review_data["totalscore"]= review_data.score1+review_data.score2+review_data.score3 
  #  review_data["pos_or_neg"] = pd.Series()
    #print(review_data[:2])
    #print(len(review_data.index))
   # for i in range(len(review_data.index)):
    #    if review_data["totalscore"][i]>11:
     #       review_data["pos_or_neg"][i] = 1
      #  elif review_data["totalscore"][i]<4:
       #     review_data["pos_or_neg"][i] = 0
        #i +=1
    #print("——————————标注结束——————————")
    #return review_data


def markwords(review_data):
    review_data["totalscore"]= review_data.score1+review_data.score2+review_data.score3 
    review_data["pos_or_neg"] = pd.Series()
    pos = review_data[review_data.totalscore>11]
    pos["pos_or_neg"]=1
    neg = review_data[review_data.totalscore<4]
    neg["pos_or_neg"] = 0
    review_data = pos.append(neg)
    print("——————————标注结束——————————")
    return review_data


In [4]:
def train_data_input(review_data):
    a1 = cutwords(review_data)
    a2 = cleanwords(a1)
    a3 = countwords(a2)
    a4 = arraywords(a2,a3) #a4.index:[reviewbody,score1,score2,score3,review_cut_words,sent]
    a5 = markwords(a4)
    return a5

def predict_data_input(review_data):
    a1 = cutwords(review_data)
    a2 = cleanwords(a1)
    a3 = countwords(a2)
    a4 = arraywords(a2,a3) #a4.index:[reviewbody,score1,score2,score3,review_cut_words,sent]
    return a4

In [5]:
a0 = reviewdata[0:20000]
a1 = cutwords(a0)
a2 = cleanwords(a1)
reviewdict = countwords(a2)
a4 = arraywords(a2,reviewdict) #a4.index:[reviewbody,score1,score2,score3,review_cut_words,sent]
a5 = markwords(a4)
traindata = a5
traindata.to_csv("traindata.csv",index=False,header=True)
#predictdata = predict_data_input(reviewdata[20000:20100])
print("traindata information————————————————————————————————————————————————————")
print(traindata[:3],traindata.columns,traindata.index)

#print("predictdata information————————————————————————————————————————————————————")
#print(predictdata[:3],predictdata.columns,predictdata.index)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/t9/186m3wkn1c1c1y1m9dmv76zh0000gn/T/jieba.cache
Loading model cost 1.610 seconds.
Prefix dict has been built succesfully.
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


——————————分词结束——————————
——————————词频统计结束——————————


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

——————————词向量化结束——————————


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pan

——————————标注结束——————————
traindata information————————————————————————————————————————————————————
                                          reviewbody  score1  score2  score3  \
4  去了好几次，德国猪肘是这里的特色菜，基本上每桌必点！还有这里的生啤也不错！如果要坐外面需要自...     4.0     4.0     4.0   
6  以前去过几次衡山路那家 新天地这家是第一次来 新装修过的果然环境很不错 有种置身国外的感觉 ...     4.0     4.0     4.0   
8                                  老饭店，味道一如既往好，价格实惠。     4.0     4.0     4.0   

                                    review_cut_words  \
4  [去, 了, 好, 几次, 德国, 猪肘, 是, 这里, 特色菜, 基本上, 每桌, 必点,...   
6  [以前, 去过, 几次, 衡山路, 那, 家, 新天地, 这家, 是, 第一次, 来, 新装...   
8                       [老, 饭店, 味道, 一如既往, 好, 价格, 实惠]   

                                                sent  totalscore  pos_or_neg  
4  [4010, 21166, 8085, 325, 20, 8, 10337, 993, 46...        12.0           1  
6  [164, 2382, 3708, 474, 3021, 20, 80, 604, 185,...        12.0           1  
8  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...        12.0           1   Index(['reviewbody', 'score1', 'score2'

In [6]:
traindata = traindata.dropna(axis=0,how='any') #去掉所有有缺失值的行
print(traindata.index)
x = np.array(list(traindata['sent']),dtype = object)[::2]  #x：训练数据的Numpy数组（如果模型有单个输入）或Numpy数组列表（如果模型有多个输入）。
y = np.array(list(traindata['pos_or_neg']),dtype = object)[::2] #y：目标（标签）数据的Numpy数组（如果模型具有单个输出）或Numpy数组列表
xt = np.array(list(traindata['sent']),dtype = object)[1::2] #测试集
yt = np.array(list(traindata['pos_or_neg']),dtype = object)[1::2]
xa = np.array(list(traindata['sent']),dtype = object) #全集
ya = np.array(list(traindata['pos_or_neg']),dtype = object)

print("————————模型构建中————————")
model = Sequential()
model.add(Embedding(len(reviewdict)+1, 256))
model.add(LSTM(output_dim=32, activation='sigmoid', inner_activation='hard_sigmoid')) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(input_dim = 32, output_dim = 1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy']) #class_mode="binary"
print("————————模型构建完成————————")
model.fit(x, y, batch_size=32, nb_epoch=5, validation_data=(xt, yt),validation_steps=None) #训练时间为若干个小时

classes = model.predict_classes(xa)
print(classes)
score = model.evaluate(xt, yt, verbose=1) #evaluate用来测试函数的性能
print ("Test accuracy = :",score)
#Keras文档使用三组不同的数据：培训数据，验证数据和测试数据。
#(training data)训练数据用于优化模型参数。(validation data)验证数据用于对元参数进行选择，例如时期的数量。
#在用最佳元参数优化模型之后，使用测试数据(test data)来获得对模型性能的合理估计。



Int64Index([    4,     6,     8,    18,    19,    23,    24,    25,    28,
               29,
            ...
            19784, 19787, 19790, 19792, 19801, 19848, 19876, 19902, 19957,
            19964],
           dtype='int64', length=9940)
————————模型构建中————————


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", units=32, recurrent_activation="hard_sigmoid")`
  del sys.path[0]
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=32, units=1)`
  from ipykernel import kernelapp as app
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


————————模型构建完成————————
Train on 4970 samples, validate on 4970 samples
Epoch 1/5
4970/4970 [==============================] - 56s 11ms/step - loss: 0.3830 - acc: 0.8463 - val_loss: 0.2498 - val_acc: 0.9189
Epoch 2/5
4970/4970 [==============================] - 48s 10ms/step - loss: 0.2408 - acc: 0.9203 - val_loss: 0.2069 - val_acc: 0.9201
Epoch 3/5
4970/4970 [==============================] - 45s 9ms/step - loss: 0.2042 - acc: 0.9229 - val_loss: 0.1965 - val_acc: 0.9213
Epoch 4/5
4970/4970 [==============================] - 45s 9ms/step - loss: 0.1805 - acc: 0.9266 - val_loss: 0.1901 - val_acc: 0.9274
Epoch 5/5
4970/4970 [==============================] - 53s 11ms/step - loss: 0.1557 - acc: 0.9394 - val_loss: 0.1796 - val_acc: 0.9324
[[1]
 [1]
 [1]
 ..., 
 [1]
 [0]
 [0]]
4970/4970 [==============================] - 7s 1ms/step
Test accuracy = : [0.17958019513119633, 0.93239436620917604]


In [10]:
b0 = reviewdata[0:200]
b1 = cutwords(a0)
b2 = cleanwords(a1)
reviewdict_b = countwords(a2)
b4 = arraywords(a2,reviewdict_b) #a4.index:[reviewbody,score1,score2,score3,review_cut_words,sent]
predictdata = b4

xb = np.array(list(predictdata['sent']),dtype = object)
predict_result = model.predict(xb, batch_size=32, verbose=0)
print("predict = :",predict_result)
print("predict type = :",type(predict_result))
p = predict_result.tolist()
predict_data = pd.DataFrame()
predict_data["prediction"]= pd.Series()
predict_data["prediction"] = p
predict_data["review_cut_words"]=predictdata["review_cut_words"]
predict_data.to_csv("predict_data.csv",index=False,header=True)


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


——————————分词结束——————————
——————————词频统计结束——————————


/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


——————————词向量化结束——————————
predict = : [[ 0.99117285]
 [ 0.89488405]
 [ 0.95861006]
 ..., 
 [ 0.9933688 ]
 [ 0.36331099]
 [ 0.98964137]]
predict type = : <class 'numpy.ndarray'>
